In [36]:
import pandas as pd
import re
import numpy as np

In [13]:
df = pd.read_csv("../raw_data/youtube_conversations.csv")

In [14]:
df = df[["transcript_id", "topic", "utterance_id", "interlocutor", "utterance_text", "main_therapist_behaviour", "client_talk_type"]]

In [63]:
def create_input_output_df(df: pd.DataFrame) -> pd.DataFrame:
    preprocessed_df = pd.DataFrame(index=np.arange(df[df.interlocutor == "therapist"].shape[0]))
    preprocessed_df["transcript_id"] = -1
    preprocessed_df["input"] = ""
    preprocessed_df["output"] = ""
    
    row_cursor = 0
    i = 0
    for transcript_id in df.transcript_id.unique():
        
        sub_df = df[df.transcript_id==transcript_id]
        # print(transcript_id, sub_df.shape)
        i_prior = i
        while i < sub_df.shape[0] + i_prior:
            client_text = ""
            while i < sub_df.shape[0] + i_prior and sub_df.interlocutor[i] == "client":
                client_text += ("; " if client_text else "") + sub_df.utterance_text[i]
                i += 1
            
            therapist_text = ""
            while i < sub_df.shape[0] + i_prior and sub_df.interlocutor[i] == "therapist":
                therapist_text += ("; " if therapist_text else "") + sub_df.utterance_text[i]
                i += 1
            if i >= sub_df.shape[0] + i_prior:
                break
            preprocessed_df["input"][row_cursor] = client_text
            preprocessed_df["output"][row_cursor] = therapist_text
            preprocessed_df["transcript_id"][row_cursor] = transcript_id
            row_cursor += 1
    return preprocessed_df[preprocessed_df.transcript_id != -1]
    

In [65]:
create_input_output_df(df)

/tmp/ipykernel_15443/644873416.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df["input"][row_cursor] = client_text
/tmp/ipykernel_15443/644873416.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df["output"][row_cursor] = therapist_text
/tmp/ipykernel_15443/644873416.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df["transcript_id"][row_cursor] = transcript_id


,transcript_id,input,output
0,0,,Thanks for filling it out. We give this form t...
1,0,Sure.,"So, let's see. It looks that you put-- You dri..."
2,0,Mm-hmm.,-and you usually have three to four drinks whe...
3,0,Usually three drinks and glasses of wine.,Okay. That's at least 12 drinks a week.
4,0,Something like that.,"Okay. Just so you know, my role, um, when we t..."
...,...,...,...
4780,133,"Yeah, like, I guess I could be wondering that ...","So you know there's a part of you going, I was..."
4781,133,Hmm.,And this is a pretty lonely road to-to walk by...
4782,133,There's Fiona.,Yeah.
4783,133,[unintelligible 00:26:37] getting out so what-...,You know maybe they'll walk that road with you...


In [67]:
input_output_df = create_input_output_df(df)
input_output_df["joined_inputs"] = ""
joined_inputs = ""
for i in range(len(input_output_df)):
    if i > 0 and input_output_df.transcript_id[i] != input_output_df.transcript_id[i-1]:
        joined_inputs = ""
    joined_inputs += (" EOS " if joined_inputs else "") + input_output_df.input[i]
    input_output_df["joined_inputs"][i] = joined_inputs
    joined_inputs += (" EOS " if joined_inputs else "") + input_output_df.output[i]
input_output_df =\
    input_output_df.drop(columns="transcript_id")\
        .applymap(lambda exp_string: re.sub(r"\[.*\]","",exp_string).strip())
input_output_df["input"] = input_output_df["joined_inputs"]
input_output_df.drop(columns="joined_inputs", inplace=True)

/tmp/ipykernel_15443/644873416.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df["input"][row_cursor] = client_text
/tmp/ipykernel_15443/644873416.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df["output"][row_cursor] = therapist_text
/tmp/ipykernel_15443/644873416.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df["transcript_id"][row_cursor] = transcript_id
/tmp/ipykernel_15443

In [68]:
input_output_df.to_csv("../preprocessed_data_level1/chats/preprocessed_youtube-conversations.csv")